<a href="https://colab.research.google.com/github/Sriharshitha0408/Mphasis-SM-Girls-Submissions/blob/main/Module_03-Database_Models/Database_Activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SQLite: Relational

SRI HARSHITHA KARE

In [ ]:
import sqlite3

db = 'museum.db'
try:
    # Connect to a database file (or create if it doesn't exist)
    connection = sqlite3.connect(db)
    cursor = connection.cursor()
    print(f"Connection and cursor to SQLite database {db} established successfully.")
except sqlite3.Error as e:
    print(f"Error connecting to database: {e}")

Connection and cursor to SQLite database museum.db established successfully.


In [ ]:
# multiline strings in Python can be sorrounded with ''' Multi line text '''
query = '''
CREATE TABLE IF NOT EXISTS exhibitions (
    exhibition_id INTEGER PRIMARY KEY,
    title TEXT,
    start_date DATE,
    end_date DATE
);
'''
cursor.execute(query)

In [ ]:
# snake_case
# cursor.execute("INSERT INTO exhibitions (exhibition_id, title, start_date, end_date) VALUES ('Ancient Greece: A Retrospective', '2025-09-01', '2026-03-30')")
# INSERT INTO exhibitions (exhibition_id, title, start_date, end_date)
# VALUES
#     (1, 'Ancient Greece: A Retrospective', '2025-09-01', '2026-03-30'),
#     (2, 'The Art of the Impressionists', '2025-10-15', '2026-01-15');
# );

data_to_insert = [
    (1, 'Ancient Greece: A Retrospective', '2025-09-01', '2026-03-30'),
    (2, 'The Art of the Impressionists', '2025-10-15', '2026-01-15')
]

cursor.executemany("INSERT INTO exhibitions(exhibition_id, title, start_date, end_date) VALUES (?, ?, ?, ?)", data_to_insert)

In [ ]:
rows = cursor.execute("SELECT * FROM exhibitions").fetchall()
for row in rows:
    print(row)

(1, 'Ancient Greece: A Retrospective', '2025-09-01', '2026-03-30')
(2, 'The Art of the Impressionists', '2025-10-15', '2026-01-15')


### Tables

The **table** is the **entity**, and the **columns** are the **attributes**. The inserted data is called **rows** or **records**.

### The Breakdown

* **Entity:** The `exhibitions` table is the entity. It represents a real-world object or concept.
* **Attributes:** The columns (`exhibition_id`, `title`, `start_date`, `end_date`) are the attributes. They define the properties of the entity.
* **Rows/Records:** Each line of data you insert is a **row** or a **record**. A single row is a specific instance of the entity. For example, the row containing `'Ancient Greece: A Retrospective'` is one record in the `Exhibitions` table. It represents one specific exhibition instance.

In SQLite, **PRIMARY KEYs**, **AUTOINCREMENT**, and **rowids** are related concepts that help uniquely identify rows, but they work together in specific ways.

**The Rowids**

Every single table in SQLite has a hidden, unique identifier called the **rowid**. It's an integer that automatically increments with each new row inserted. Think of it as the table's built-in, default primary key. It's always there, even if you don't explicitly define a primary key.

**Primary Key**

When you define a column as `PRIMARY KEY`, it becomes the official, explicit way to identify rows. This is an essential concept for database design because it's a **constraint** that guarantees uniqueness and non-null values for that column.

There are two types of primary keys in SQLite:

1.  **Integer Primary Key:** If you define an `INTEGER` column as the `PRIMARY KEY`, that column becomes an **alias** for the hidden **rowid**. This is the most common and efficient way to create an auto-incrementing key in SQLite. When you insert a new row and don't provide a value for this column, SQLite automatically assigns the next available rowid.

2.  **Other Primary Keys:** If you define any other data type (like `TEXT` or `REAL`) or a composite key as the `PRIMARY KEY`, SQLite creates a regular index on that column. The hidden **rowid** still exists and is used internally, but your chosen primary key is the official identifier for the table.

**AUTOINCREMENT**

The `AUTOINCREMENT` keyword in SQLite is a specific instruction that goes with an `INTEGER PRIMARY KEY`. It adds an extra layer of behavior: it guarantees that the new rowid will be **strictly greater** than any previously used rowid.

This is different from the default behavior of an `INTEGER PRIMARY KEY` alone. Without `AUTOINCREMENT`, if you were to delete the highest-numbered row, the next inserted row might reuse that now-available rowid. With `AUTOINCREMENT`, this never happens; the counter just keeps going up.

It's important to note that `AUTOINCREMENT` is generally **not necessary** for most applications. The default behavior of a simple `INTEGER PRIMARY KEY` is usually sufficient and more efficient. The `AUTOINCREMENT` keyword should only be used if you have a specific requirement to ensure that IDs are never reused and always increase, even after deletions.

## CRUD

In [ ]:
def read_exhibitions():
    """Retrieves and prints all exhibitions."""
    print("\nCurrent Exhibitions:")
    cursor.execute('SELECT * FROM Exhibitions')
    rows = cursor.fetchall()
    for row in rows:
        print(f"ID: {row[0]}, Title: {row[1]}, Start: {row[2]}, End: {row[3]}")

def read_single_exhibition(exhibition_id):
    """Retrieves and prints a single exhibition by its ID."""
    print(f"\nRetrieving exhibition with ID {exhibition_id}:")
    cursor.execute('SELECT * FROM exhibitions WHERE exhibition_id = ?', (exhibition_id,))
    row = cursor.fetchone()
    if row:
        print(f"ID: {row[0]}, Title: {row[1]}, Start: {row[2]}, End: {row[3]}")
    else:
        print("Exhibition not found.")

def update_exhibition_dates(exhibition_id, new_end_date):
    """Updates the end date for a specific exhibition."""
    cursor.execute('''
        UPDATE exhibitions
        SET end_date = ?
        WHERE exhibition_id = ?
    ''', (new_end_date, exhibition_id))
    connection.commit()
    print(f"\nUpdated exhibition with ID {exhibition_id}. New end date: {new_end_date}")

def delete_exhibition(exhibition_id):
    """Deletes an exhibition from the table."""
    cursor.execute('DELETE FROM exhibitions WHERE exhibition_id = ?', (exhibition_id,))
    connection.commit()
    print(f"\nDeleted exhibition with ID {exhibition_id}.")


In [ ]:
update_exhibition_dates(1, '2026-04-30')
read_single_exhibition(1)


Updated exhibition with ID 1. New end date: 2026-04-30

Retrieving exhibition with ID 1:
ID: 1, Title: Ancient Greece: A Retrospective, Start: 2025-09-01, End: 2026-04-30


In [ ]:
delete_exhibition(2)
read_exhibitions()

# Close the connection when done
connection.close()
print("\nDatabase connection closed.")


Deleted exhibition with ID 2.

Current Exhibitions:
ID: 1, Title: Ancient Greece: A Retrospective, Start: 2025-09-01, End: 2026-04-30

Database connection closed.


## Redis: Key-Value

In [ ]:
!apt-get update
!apt-get install redis-server -y

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
!pip install redis

In [ ]:
!service redis-server start

Starting redis-server: redis-server.


In [ ]:
import redis

# Connect to the local Redis server
r = redis.Redis(host='localhost', port=6379, db=0)

# Test the connection with the PING command
try:
    r.ping()
    print("Redis server is up and running!")
except redis.exceptions.ConnectionError as e:
    print(f"Connection error: {e}")

# Set and get a key-value pair
r.set('name', 'Alice')
name = r.get('name').decode('utf-8')
print(f"Retrieved name: {name}")

Redis server is up and running!
Retrieved name: Alice


In [ ]:
import redis

r = redis.Redis(host='localhost', port=6379, db=0)

exhibition_data = {
    'title': 'Ancient Greece: A Retrospective',
    'start_date': '2025-09-01',
    'end_date': '2026-03-30',
    'curator': 'Jane Doe'  # An attribute that can be added easily
}

r.hset('exhibition:101', mapping=exhibition_data)

# Retrieve all fields and values from the hash
all_fields = r.hgetall('exhibition:101')
decoded_fields = {key.decode('utf-8'): value.decode('utf-8') for key, value in all_fields.items()}

print(f"\nExhibition Data for 'exhibition:101':")
print(decoded_fields)

# To demonstrate flexibility, let's add a new field to a different exhibition
# This doesn't require schema changes for other records
r.hset('exhibition:102', 'theme', 'Impressionist paintings')

# Retrieve the new exhibition's data
new_exhibition_data = r.hgetall('exhibition:102')
decoded_new_data = {key.decode('utf-8'): value.decode('utf-8') for key, value in new_exhibition_data.items()}

print(f"\nExhibition Data for 'exhibition:102':")
print(decoded_new_data)


Exhibition Data for 'exhibition:101':
{'title': 'Ancient Greece: A Retrospective', 'start_date': '2025-09-01', 'end_date': '2026-03-30', 'curator': 'Jane Doe'}

Exhibition Data for 'exhibition:102':
{'theme': 'Impressionist paintings'}


### Collections

**exhibition** is the name of the **collection**, and **101** and **102** are the unique **IDs**.


### Explanation

In the Redis code, the key for each record is a compound string like `exhibition:101` and `exhibition:102`.

* **`exhibition`**: This is the prefix or namespace. It serves a purpose similar to a table name in a relational database—it groups related items together, forming a logical **collection** of exhibitions.
* **`:`**: This colon is a common convention in Redis for separating the namespace from the unique ID. It helps organize your keys and makes it easy to search for all keys within a specific collection (e.g., all `exhibition:*` keys).
* **`101` and `102`**: These are the unique identifiers or **IDs** for each individual exhibition record. They function just like a `PRIMARY KEY` in a relational table, ensuring that each exhibition can be retrieved and referenced uniquely.

This structure allows the developer to organize data logically while maintaining the flexibility and simplicity of a key-value store.

In [ ]:
import json

all_exhibitions = []
r_cursor = '0'

while r_cursor != 0:
    r_cursor, keys = r.scan(cursor=r_cursor, match='exhibition:*')
    for key in keys:
        # HGETALL retrieves all fields and values for a hash
        exhibition_data = r.hgetall(key)
        # Decode the bytes and add to the list
        decoded_data = {
            'id': key.decode('utf-8').split(':')[-1],
            'attributes': {k.decode('utf-8'): v.decode('utf-8') for k, v in exhibition_data.items()}
        }
        all_exhibitions.append(decoded_data)

print("Retrieved the entire 'exhibition' collection:")
print(json.dumps(all_exhibitions, indent=2))

Retrieved the entire 'exhibition' collection:
[
  {
    "id": "101",
    "attributes": {
      "title": "Ancient Greece: A Retrospective",
      "start_date": "2025-09-01",
      "end_date": "2026-03-30",
      "curator": "Jane Doe"
    }
  },
  {
    "id": "102",
    "attributes": {
      "theme": "Impressionist paintings"
    }
  }
]


In Redis and MongoDB, **collections** would be the entity. While their data models are different from the relational model, the concept of a collection serves the same purpose as a table: to group a set of related objects (documents or keys) that represent a single real-world entity.

**Redis**
In Redis, a logical collection is a group of keys that share a common prefix, like `exhibition:`. This prefix acts as a namespace, functionally creating a collection. Each unique key within that collection, such as `exhibition:101`, represents a single instance of the entity.

**MongoDB**
In MongoDB, the term **collection** is an explicit part of the data model. A collection is a grouping of JSON-like documents. Each document in the `exhibitions` collection represents a single instance of the exhibition entity.

## MongoDB

In [ ]:
# !pip install pymongo

In [ ]:
# from pymongo import MongoClient

# # Connect to the local MongoDB server
# client = MongoClient('mongodb://localhost:27017/')

# # Select the database and collection
# db = client['museum_db']
# exhibitions_collection = db['exhibitions']

# print("Connected to MongoDB.")

In [ ]:
# # Insert one document
# one_exhibition = {
#     'title': 'Ancient Greece: A Retrospective',
#     'start_date': '2025-09-01',
#     'end_date': '2026-03-30',
#     'curator': 'Jane Doe'
# }
# result_one = exhibitions_collection.insert_one(one_exhibition)
# print(f"Inserted one document with ID: {result_one.inserted_id}")

# # Insert multiple documents
# many_exhibitions = [
#     {
#         'title': 'The Art of the Impressionists',
#         'start_date': '2025-10-15',
#         'end_date': '2026-01-15'
#     },
#     {
#         'title': 'Dinosaurs: The Untold Story',
#         'start_date': '2026-05-01',
#         'end_date': '2026-11-01',
#         'theme': 'Paleontology'
#     }
# ]
# result_many = exhibitions_collection.insert_many(many_exhibitions)
# print(f"Inserted multiple documents with IDs: {result_many.inserted_ids}")

In [ ]:
# # Find all documents
# print("\nFinding all documents:")
# for doc in exhibitions_collection.find({}):
#     print(doc)

# # Find documents with specific criteria
# print("\nFinding documents curated by 'Jane Doe':")
# query = {'curator': 'Jane Doe'}
# for doc in exhibitions_collection.find(query):
#     print(doc)

In [ ]:
# # Update a document by adding a new field
# update_query = {'title': 'The Art of the Impressionists'}
# new_values = {'$set': {'curator': 'John Smith'}}
# exhibitions_collection.update_one(update_query, new_values)

# print("\nUpdated 'The Art of the Impressionists' with a curator.")
# updated_doc = exhibitions_collection.find_one({'title': 'The Art of the Impressionists'})
# print(updated_doc)

In [ ]:
# # Delete a specific document
# delete_query = {'title': 'Ancient Greece: A Retrospective'}
# result = exhibitions_collection.delete_one(delete_query)

# print(f"\nDeleted {result.deleted_count} document.")

# # Verify deletion
# print("\nChecking remaining documents:")
# for doc in exhibitions_collection.find({}):
#     print(doc)